In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import datetime
from datetime import timedelta
import pymysql
from sqlalchemy import create_engine
# 导入模型
import xgboost as xgb

In [3]:
mysql_setting = {
        'host': '47.100.201.211',
        'port': 3306,
        'user': 'root',
        'passwd': 'iyGfLR64Ne4Ddhk7',
        # 数据库名称
        'db': 'data',
        'charset': 'utf8'
    }
engine = create_engine(
        "mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(
            **mysql_setting),
        max_overflow=5)

In [3]:
# sql = 'TRUNCATE TABLE predictAdvice'
# result = engine.execute(sql)
# table_name = 'predictAdvice'
# advice.to_sql(table_name, engine, if_exists='append', index=False)

# 接口

In [8]:
def predict(product, span):
    main = product
    # 通过sqlalchemy库连接mysql
    mysql_setting = {
            'host': '47.100.201.211',
            'port': 3306,
            'user': 'root',
            'passwd': 'iyGfLR64Ne4Ddhk7',
            # 数据库名称
            'db': 'data',
            'charset': 'utf8'
        }
    engine = create_engine("mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(**mysql_setting),max_overflow=5)
    # advice和relation表共用先获取
    sql_cmd = "select * from predictAdvice1"
    advice1 = pd.read_sql(sql_cmd, engine)
    sql_cmd = "select * from predictAdvice2"
    advice2 = pd.read_sql(sql_cmd, engine)
    sql_cmd = "select * from productRelation where product = %(product)s"
    relation = pd.read_sql(sql_cmd, engine, params={'product': product})
    # 首选农产品价格预测
    def mainPredict(product, span):
        mainproduct = {}
        mainproduct["name"] = product
        # 判断是长期预测还是短期预测
        if span == 0:
            mainproduct["span"] = 0
            sql_cmd = "select * from predict_online_short where product = %(product)s"
            data_online = pd.read_sql(sql_cmd, engine, params={'product': product})
            sql_cmd = "select * from predict_offline_short where product = %(product)s"
            data_offline = pd.read_sql(sql_cmd, engine, params={'product': product})
        elif span == 1:
            mainproduct["span"] = 1
            sql_cmd = "select * from predict_online_long where product = %(product)s"
            data_online = pd.read_sql(sql_cmd, engine, params={'product': product})
            sql_cmd = "select * from predict_offline_long where product = %(product)s"
            data_offline = pd.read_sql(sql_cmd, engine, params={'product': product})
        price_predict = [] # 存放所有线上线下价格
    # 判断是否有线上价格和线下价格
        # 有线下价格
        if data_offline.empty and data_online.empty:
            return False
        if not data_offline.empty: 
            mainproduct["online"] = 0
            # pricelist[{"date": , "predict price":}]
            df_offline = []
            df1 = data_offline.rename(columns={"price":"predict_price"})
            df2 = df1[["date", "predict_price"]].to_dict('index')
            for i in df2.values():
                df_offline.append(i)
            mainproduct["pricelist"] = df_offline
            price_predict.append(mainproduct)
        # 有线上价格
        if not data_online.empty: 
            mainproduct["online"] = 1
            if mainproduct["span"] == 0:
                mainproduct["consumer_advice"] = advice1.loc[advice1['product']==product]['consumer advice'].values[0]
            else:
                mainproduct["farmer_advice"] = advice1.loc[advice1['product']==product]['farmer advice'].values[0]
            # pricelist价格
            df_online = [] 
            df3 = data_online.rename(columns={"price":"predict_price"})
            df4 = df3[["date", "predict_price"]].to_dict('index')
            for i in df4.values():
                df_online.append(i)
            mainproduct["pricelist"] = df_online
            price_predict.append(mainproduct)
        return price_predict


    def predictOther(product, span):
        mainproduct = {}
        mainproduct["name"] = product
        # 判断是长期预测还是短期预测
        if span == 0:
            mainproduct["span"] = 0
            sql_cmd = "select * from predict_online_short where product = %(product)s"
            data_online = pd.read_sql(sql_cmd, engine, params={'product': product})
            sql_cmd = "select * from predict_offline_short where product = %(product)s"
            data_offline = pd.read_sql(sql_cmd, engine, params={'product': product})
        elif span == 1:
            mainproduct["span"] = 1
            sql_cmd = "select * from predict_online_long where product = %(product)s"
            data_online = pd.read_sql(sql_cmd, engine, params={'product': product})
            sql_cmd = "select * from predict_offline_long where product = %(product)s"
            data_offline = pd.read_sql(sql_cmd, engine, params={'product': product})
        price_predict_other  = [] # 存放所有线上线下价格
        # 有线下价格
        if not data_offline.empty: 
            mainproduct["online"] = 0            
            df_offline = []
            df1 = data_offline.rename(columns={"price":"predict_price"})
            df2 = df1[["date", "predict_price"]].to_dict('index')
            for i in df2.values():
                df_offline.append(i)
            mainproduct["pricelist"] = df_offline
            price_predict_other.append(mainproduct)
            # 有线上价格
        if not data_online.empty: 
            mainproduct["online"] = 1
            if mainproduct["span"] == 0:
                mainproduct["seller_advice"] = advice2.loc[(advice2['main']==main)&(advice2['cp']==product), 'seller advice'].values[0]
            # pricelist价格
            df_online = [] 
            df3 = data_online.rename(columns={"price":"predict_price"})
            df4 = df3[["date", "predict_price"]].to_dict('index')
            for i in df4.values():
                df_online.append(i)
            mainproduct["pricelist"] = df_online
            mainproduct
            price_predict_other.append(mainproduct)
        return price_predict_other    
    
    # 竞品（cp）价格预测
    def cpPredict(product, span):
        cp_price_predict = []
        # 获取竞品关系(先判断有无竞品)
        if relation["cpProduct"][0] is not None:
            cplist = [x for x in relation["cpProduct"][0].split()]
            for cpProduct in cplist:
                for i in predictOther(cpProduct, span):
                    cp_price_predict.append(i)
        return cp_price_predict
    # 替代品（sub）价格预测
    def subPredict(product, span):
        sub_price_predict = []
        # 获取替代品关系(先判断有无替代品)
        if relation["subProduct"][0] is not None:
            sublist = [x for x in relation["subProduct"][0].split()]
            for subProduct in sublist:
                for i in predictOther(subProduct, span):
                    sub_price_predict.append(i)
        return sub_price_predict
    def getPrice(product, span):
        data = {}
        data["price_predict"] = []
        data["cp_price_predict"] = []
        data["sub_price_predict"] = []
        for i in mainPredict(product, span):
            data["price_predict"].append(i)
        for i in cpPredict(product, span):
            data["cp_price_predict"].append(i)
        for i in subPredict(product, span):
            data["sub_price_predict"].append(i)
        return data
    return getPrice(product, span)

In [11]:
def comparePrice(product, start, end):
    # 通过sqlalchemy库连接mysql
    mysql_setting = {
            'host': '47.100.201.211',
            'port': 3306,
            'user': 'root',
            'passwd': 'iyGfLR64Ne4Ddhk7',
            # 数据库名称
            'db': 'data',
            'charset': 'utf8'
        }
    engine = create_engine("mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(**mysql_setting),max_overflow=5)
    sql_cmd = "select * from productRelation where product = %(product)s"
    relation = pd.read_sql(sql_cmd, engine, params={'product': product})   
    def mainCompare(product):
        df_all = []
        main_product_compare = {}
        main_product_compare['name'] = product
        if (len(start) == 10 and len(end) == 10):
            main_product_compare['span'] = 0
            sql_cmd = "select * from price_real_predict_compare_short where product = %(product)s and date >= %(start)s and date <= %(end)s"
        elif (len(start) == 7 and len(end) == 7):
            main_product_compare['span'] = 1
            sql_cmd = "select * from price_real_predict_compare_long where product = %(product)s and date >= %(start)s and date <= %(end)s"
        else:
            return False
        dt = pd.read_sql(sql_cmd, engine, params={'product': product, 'start':start, 'end':end})
        dt = dt.fillna('')
        df_online = [] 
        dt1 = dt[["date", "real_price", "predict_price"]].to_dict('index')
        for i in dt1.values():
            df_online.append(i)
        main_product_compare["pricelist"] = df_online
        df_all.append(main_product_compare)
        return df_all
    def cpCompare(product):
        cp_product_compare = []
        # 获取竞品关系(先判断有无竞品)
        if relation["cpProduct"][0] is not None:
            cplist = [x for x in relation["cpProduct"][0].split()]
            for cpProduct in cplist:
                for i in mainCompare(cpProduct):
                    cp_product_compare.append(i)
        return cp_product_compare
    def getAll(product):
        data = {}
        data["main_product_compare"] = []
        data["cp_product_compare"] = []
        for i in mainCompare(product):
            data["main_product_compare"].append(i)
        for i in cpCompare(product):
            data["cp_product_compare"].append(i)
        return data
    return getAll(product)

{'main_product_compare': [{'name': '巢湖中垾番茄',
   'span': 0,
   'pricelist': [{'date': datetime.date(2022, 7, 1),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 2),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 3),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 4),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 5),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 6),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 7),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 8),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 9),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 10),
     'real_price': 9.93,
     'predict_price': ''},
    {'d

In [39]:
comparePrice('巢湖中垾番茄', '2022-07-15', '2022-08-15')

{'main_product_compare': [{'name': '巢湖中垾番茄',
   'span': 0,
   'pricelist': [{'date': datetime.date(2022, 7, 15),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 16),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 17),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 18),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 19),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 20),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 21),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 22),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 23),
     'real_price': 9.93,
     'predict_price': ''},
    {'date': datetime.date(2022, 7, 24),
     'real_price': 9.93,
     'predict_price': ''}

In [9]:
predict('仙桃香米', 1)

{'price_predict': [{'name': '仙桃香米',
   'span': 1,
   'online': 1,
   'farmer_advice': '湖北仙桃洪茂春春红米一般采用传统人工育苗、插秧等种植方式，一年一季慢生长，在5月份播种，9月下旬收获，约105天生长期，有利于大米营养物质的积累。另外，通过21道工序筛选严格品控，为了让客户食用到新鲜好米，洪茂春粮食从稻田到餐桌完整产业链，多道程序严苛把控，确保了每一粒大米的品质。预测未来一年湖北仙桃洪茂春春红米价格将会降低，由于大米种植期较长，因此建议适当减少种植面积，控制生产成本，并有效利用空余土地生产其他湖北仙桃特有的高价值作物例如湖北新鲜泥巴藕等。',
   'pricelist': [{'date': datetime.date(2022, 9, 1), 'predict_price': 4.8},
    {'date': datetime.date(2022, 10, 1), 'predict_price': 4.8},
    {'date': datetime.date(2022, 11, 1), 'predict_price': 4.8},
    {'date': datetime.date(2022, 12, 1), 'predict_price': 4.8}]}],
 'cp_price_predict': [{'name': '万年贡蟹田丝苗米',
   'span': 1,
   'online': 1,
   'pricelist': [{'date': datetime.date(2022, 9, 1), 'predict_price': 4.92},
    {'date': datetime.date(2022, 10, 1), 'predict_price': 4.91},
    {'date': datetime.date(2022, 11, 1), 'predict_price': 4.93},
    {'date': datetime.date(2022, 12, 1), 'predict_price': 4.85}]},
  {'name': '巢湖稻虾米',
   'span': 1,
   'online': 1,
   'pricelis